# Hypothesis Testing on IFSC and 8a.nu Climbing Data

## Introduction

This section explores statistical relationships between outdoor and competition climbing performance using a merged dataset derived from two key sources: **IFSC** (competition climbing data) and **8a.nu** (outdoor climbing data), which was scraped and merged in the notebooks located in the `01_data_collection` folder. For details on the data collection and merging process, refer to those notebooks. By analyzing this unified dataset, we aim to investigate whether measurable trends or correlations exist between climbers’ achievements across different contexts.

## Objectives

The main goals of this analysis are to test the following hypotheses:

1. **Outdoor vs. Competition Performance**
   Do climbers with stronger outdoor climbing metrics—such as higher maximum grade or a greater number of 8c+ ascents—also perform better in competitions?
   *Separate hypothesis tests will be conducted for each competition discipline: bouldering, lead, and combined.*

2. **Interdisciplinary Competition Correlation**
   Is there a statistically significant correlation between performance in different IFSC competition disciplines (e.g., do high lead scores correlate with high boulder scores)?

3. **Outdoor Performance Correlation**
   Within outdoor climbing, is there a meaningful correlation between the metrics (e.g., do higher average grades correlate with higher best grade climbed)?


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv('../data/final_data.csv')

## Hypothesis Test: Outdoor Grade 8c+ vs. Competition Performance

Do climbers perform better in **competitions** if they have climbed an **outdoor grade of 8c+ or above**?

### Methodology

- **Test Type:** One-tailed, two-sample t-test
- **Grouping Variable:** Whether the climber has climbed 8c+ or above
- **Dependent Variable:** Competition points (tested separately for 3 disciplines)

This test aims to determine whether there is a statistically significant difference in competition performance between climbers who have reached the 8c+ outdoor benchmark and those who have not.



### Test 1: Outdoor Grade 8c+ vs **Bouldering**
Do climbers perform better in **bouldering competitions** if they have climbed an **outdoor grade of 8c+ or above**?

#### Hypotheses

- **Null Hypothesis (H₀):**  Climbing an outdoor grade of 8c+ or above has no effect on bouldering competition performance.
  Mathematically:
  μ₁ = μ₂  *(Where μ₁ = mean boulder points for climbers who have climbed 8c+ or above, μ₂ = mean boulder points for those who have not.)*

- **Alternative Hypothesis (Hₐ):**  Climbers who have climbed an outdoor grade of 8c+ or above perform better in bouldering competitions.
  Mathematically:
  μ₁ > μ₂